In [1]:
paragraph_1 = """
Durefishan Saleem (born 03 January 1996) is a Pakistani actress of Punjabi descent who appears on Urdu television.
She made her acting debut in 2020 with a supporting role in the drama serial Dil Ruba and soon went on to star in Bhaaras,
where she received an ARY People’s Choice Award. 
Born in Lahore, Punjab, to Saleem-ul-Hassan (a director and producer for PTV in the 2000s),
she obtained her LLB from University College London before moving to Karachi in 2019 to pursue acting.
Her performances in Pardes (with Affan Waheed) and Juda Huay Kuch Is Tarha further established her,
and she gained wider recognition with her leading role in Kaisi Teri Khudgarzi (2022) opposite Danish Taimoor,
followed by Ishq Murshid (2023). 
Durefishan also appeared in Jurm alongside Wahaj Ali, the anthology series Siyaah, and the social drama Jaisay Aapki Marzi 
opposite Mikaal Zulfiqar. She was reportedly offered a role in Jhok Sarkar with Farhan Saeed, portrayed Shibra Sulaiman in 
Ishq Murshid opposite Bilal Abbas Khan, starred in the 2024 revenge-thriller Khaie with Faysal Quraishi, and is set to appear 
in Sanwal Yaar Piya (2025) with Feroze Khan and Ahmed Ali Akbar, marking their first on-screen collaboration.
"""

paragraph_2 = """
Bilal Abbas Khan, born on 4 June 1993 in Karachi to a Pathan family, discovered his passion for acting at Lawrence College in Murree,
later earning a degree in marketing from the Shaheed Zulfikar Ali Bhutto Institute of Science and Technology before formally studying
acting at the National Academy of Performing Arts. Known for his boy-next-door portrayals, Bilal gained recognition through projects
like *Dumpukht - Aatish-e-Ishq*, *O Rangreza*, *Balaa*, *Cheekh*, and the telefilm *Laal*, eventually solidifying his star status with
hits such as *Pyar Ke Sadqay*, *Dobara*, and *Khel Khel Mein*, winning three Lux Style Awards along the way. Meanwhile, Durefishan Saleem,
born on 03 January 1996 in Lahore, comes from a creative background—her father Saleem-ul-Hassan was a director and producer for PTV—and
she earned an LLB from University College London before moving to Karachi in 2019 to pursue acting. Following her 2020 supporting debut
in *Dil Ruba*, she won an ARY People’s Choice Award for *Bhaaras* and went on to star in *Pardes*, *Juda Huay Kuch Is Tarha*, 
*Kaisi Teri Khudgarzi*, *Jurm*, *Siyaah*, *Jaisay Aapki Marzi*, and the 2024 revenge-thriller *Khaie*. In 2023, Bilal and Durefishan 
joined forces in *Ishq Murshid*, a romantic, light-hearted series directed by Farooq Rind and produced by Moomal Entertainment and 
MD Productions. Despite mixed reviews for its plot and characters, the show earned a record-breaking TRP rating of 22.3 on Hum TV,
further propelling both actors’ careers and sparking speculation about a potential second season.
"""

In [ ]:
!pip install -q ollama pydantic

In [11]:
import ollama
from pydantic import BaseModel, Field, ValidationError
from typing import List

In [3]:
class Person(BaseModel):
    name: str = Field(description="The full name of the person.")
    birthday: str = Field(description="The birthday of a person, e.g., 'DD-MM-YYYY' or any recognized date format.")

In [12]:
class PersonList(BaseModel):
    persons: List[Person] = Field(description="A list of persons, each having a name and birthday.")

In [14]:
PersonList.model_json_schema()

{'$defs': {'Person': {'properties': {'name': {'description': 'The full name of the person.',
     'title': 'Name',
     'type': 'string'},
    'birthday': {'description': "The birthday of a person, e.g., 'DD-MM-YYYY' or any recognized date format.",
     'title': 'Birthday',
     'type': 'string'}},
   'required': ['name', 'birthday'],
   'title': 'Person',
   'type': 'object'}},
 'properties': {'persons': {'description': 'A list of persons, each having a name and birthday.',
   'items': {'$ref': '#/$defs/Person'},
   'title': 'Persons',
   'type': 'array'}},
 'required': ['persons'],
 'title': 'PersonList',
 'type': 'object'}

In [15]:
def extract_info(text):
    response = ollama.chat(
        model="llama3.2",
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that extracts a list of persons' names and birthdays from a given paragraph."
            },
            {
                "role": "user",
                "content": text
            }
        ],
        options = {"temperature": 0.0},
        format = PersonList.model_json_schema()
    )

    try:
        person_info = PersonList.model_validate_json(response.message.content)
        return person_info
    except ValidationError as e:
        print(f"Error: {e}")
        return None

In [16]:
p2_res = extract_info(paragraph_2)

In [17]:
p2_res

PersonList(persons=[Person(name='Bilal Abbas Khan', birthday='4 June 1993'), Person(name='Durefishan Saleem', birthday='03 January 1996')])

In [18]:
type(p2_res)

__main__.PersonList

In [22]:
for per in p2_res.persons:
    print(type(per))

<class '__main__.Person'>
<class '__main__.Person'>


In [23]:
type(p2_res.persons)

list